##
## House Prices: Advanced Regression Techniques

#### Importing Libraries

In [1]:
# Importing libraries
import warnings

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, \
    make_scorer, f1_score, recall_score, precision_score

# plot_confusion_matrix
# from sklearn.metrics import plot_confusion_matrix

# To build model for prediction
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

import statsmodels.api as sm

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


#### Full Data, and Data Cleaning
#### data null values

In [5]:
full = train.merge(test, how='outer')

In [6]:
# count null values by columns
full.isnull().sum().sort_values(ascending=False)

PoolQC          2909
MiscFeature     2814
Alley           2721
Fence           2348
SalePrice       1459
                ... 
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
HeatingQC          0
Length: 81, dtype: int64

In [11]:
# count null values by rows
# fill the null value in PoolQC
full.PoolQC.fillna('NA', inplace=True)

In [13]:
# fill the null value in MiscFeature
full.MiscFeature.value_counts()
full.MiscFeature.fillna('NA', inplace=True)

In [15]:
# fill the null value in Alley
full.Alley.value_counts()
full.Alley.fillna('NA', inplace=True)

In [17]:
# fill the null value in Fence
full.Fence.value_counts()
full.Fence.fillna('NA', inplace=True)

In [19]:
# fill the null value in FireplaceQu
full.FireplaceQu.value_counts()
full.FireplaceQu.fillna('NA', inplace=True)

In [21]:
# fill the null value in LotFrontage
full.LotFrontage.value_counts()
full.LotFrontage.fillna(full.LotFrontage.mean(), inplace=True)

In [23]:
# fill the null value in GarageCond
full.GarageCond.value_counts()
full.GarageCond.fillna('NA', inplace=True)

In [24]:
# fill the null value in GarageQual
full.GarageQual.value_counts()
full.GarageQual.fillna('NA', inplace=True)

In [25]:
# fill the null value in GarageYrBlt
full.GarageYrBlt.value_counts()
full.GarageYrBlt.fillna('NA', inplace=True)

In [26]:
# fill the null value in GarageFinish
full.GarageFinish.value_counts()
full.GarageFinish.fillna('NA', inplace=True)

In [27]:
# fill the null value in GarageType
full.GarageType.value_counts()
full.GarageType.fillna('NA', inplace=True)